In [4]:
import json
import model
import redis


[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-07-20 14:47:47.818194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 14:47:47.818216: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Connect to Redis

In [7]:
# Connect to Redis
db = redis.Redis(
    host="0.0.0.0", 
    port="6379", 
    db=0
)


In [8]:
db.ping()

True

## Read a message from the Redis queue

In [37]:
queue_name, msg = db.brpop("job")

queue_name, msg

(b'job',
 b'{"id": "26670ba3-e3fc-459e-8cdb-60ce4515880d", "text": "Metra - Speaker Connector for Select Volkswagen VehiclesCompatible with select Volkswagen vehicles; connects a speaker to the vehicle\'s stereo; easy installation "}')

If the output from Redis is a Pyton dict converted as string, we can recover it back using `json.loads`.

In [38]:
msg = json.loads(msg)
msg

{'id': '26670ba3-e3fc-459e-8cdb-60ce4515880d',
 'text': "Metra - Speaker Connector for Select Volkswagen VehiclesCompatible with select Volkswagen vehicles; connects a speaker to the vehicle's stereo; easy installation "}

In [39]:
msg['text']

"Metra - Speaker Connector for Select Volkswagen VehiclesCompatible with select Volkswagen vehicles; connects a speaker to the vehicle's stereo; easy installation "

In [40]:
out = model.modelFuction(msg['text'])

In [41]:
out

['Car Electronics & GPS',
 'Car Installation Parts & Accessories',
 'Car Audio Installation Parts']

In [18]:
pred_class, pred_score = predict(msg["image_name"])
pred_class, pred_score

('Cat', 0.9999)

In [42]:
output_msg = {
    "prediction": out
    
}

output_msg

{'prediction': ['Car Electronics & GPS',
  'Car Installation Parts & Accessories',
  'Car Audio Installation Parts']}

In [30]:
db.set(msg["id"], json.dumps(output_msg))


True

In [31]:
msg["id"]

'3aa38b71-2971-4601-81da-3c4c6eb0b42e'

In [34]:
a = json.loads(db.get("3aa38b71-2971-4601-81da-3c4c6eb0b42e"))

In [43]:
a

{'prediction': ['Computers & Tablets']}

In [44]:
while True:
    if db.get("3aa38b71-2971-4601-81da-3c4c6eb0b42e"):
        print('funciono')

funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
funciono
f

KeyboardInterrupt: 

## Store a message in Redis using a key

In [14]:
db.set("0", "It's a dog!")


True

After we add something to Redis using `set()`, that `(key, value)` pair will remain stored unless we remove it:

In [6]:
db.get("0")

In [ ]:
db.get("0")

In [ ]:
db.delete("0")

In [25]:
a = db.get('3aa38b71-2971-4601-81da-3c4c6eb0b42e')

In [ ]:
json.loads(a)

In [7]:
import pandas as pd
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

2022-07-26 12:06:00.967053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-26 12:06:00.967083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [55]:
df = pd.read_csv('DataNew.csv',index_col=0)

In [44]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['Text'])

sum = 0
for i in range(len(sequences)):
  sum = sum + len(sequences[i])

avg_length = int(sum/len(sequences))

In [33]:
num_classes = df.drop('Text',axis=1).shape[1]
print(num_classes)
max_words = len(word_index) + 1
print(max_words)
maxlen = avg_length
print(avg_length)

853
20558
23


In [47]:
padded[0]

array([3080, 1659,   26,   60,    2,   15,  114,   50, 1659,  162, 4489,
          9, 2668,   25,   60,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [45]:
padded = pad_sequences(sequences, padding="post")

In [38]:
model = Sequential()
model.add(Embedding(max_words, 300,input_length=maxlen)) # Turns positive integers (indexes) into dense vectors of fixed size, input_dim = 500, output_dim = 300
model.add(tf.keras.layers.Dense(300, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(optimizer=Adam(0.015), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])
callbacks = [
    ReduceLROnPlateau(), # Reduce learning rate when a metric has stopped improving
    ModelCheckpoint(filepath='model-neural-net.h5', save_best_only=True)
]

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 23, 300)           6167400   
                                                                 
 dense_7 (Dense)             (None, 23, 300)           90300     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 853)               256753    
                                                                 
Total params: 6,514,453
Trainable params: 6,514,453
Non-trainable params: 0
_________________________________________________________________


In [41]:
df.dtypes

Text                                    object
Connected Home & Housewares            float64
Carfi Instore Only                     float64
Car Electronics & GPS                  float64
In-Store Only                          float64
                                        ...   
All Sheet Music & Videos               float64
Labels & Stickers                      float64
Mobile Broadband Devices with Plans    float64
PS Vita Consoles                       float64
NumberCategories                         int64
Length: 854, dtype: object

In [51]:
Y = df.drop('Text',axis=1)

In [48]:
history = model.fit(padded, ,
                    epochs=30,
                    batch_size=32,
                    validation_split=0.3,
                    callbacks=callbacks)

2022-07-26 12:45:25.164180: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 243644096 exceeds 10% of free system memory.


Epoch 1/30


ValueError: in user code:

    File "/home/martin/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/martin/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/martin/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/martin/.local/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/martin/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/martin/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise Va